In [11]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

df = pd.read_csv('Top5000Albums.csv')
df

,Ranking,Album,Artist Name,Release Date,Genres,Descriptors,Average Rating,Number of Ratings,Number of Reviews
0,1.0,OK Computer,Radiohead,16 June 1997,"Alternative Rock, Art Rock","melancholic, anxious, futuristic, alienation, ...",4.23,"70,382",1531
1,2.0,Wish You Were Here,Pink Floyd,12 September 1975,"Progressive Rock, Art Rock","melancholic, atmospheric, progressive, male vo...",4.29,"48,662",983
2,3.0,In the Court of the Crimson King,King Crimson,10 October 1969,"Progressive Rock, Art Rock","fantasy, epic, progressive, philosophical, com...",4.30,"44,943",870
3,4.0,Kid A,Radiohead,3 October 2000,"Art Rock, Experimental Rock, Electronic","cold, melancholic, futuristic, atmospheric, an...",4.21,"58,590",734
4,5.0,To Pimp a Butterfly,Kendrick Lamar,15 March 2015,"Conscious Hip Hop, West Coast Hip Hop, Jazz Rap","political, conscious, poetic, protest, concept...",4.27,"44,206",379
...,...,...,...,...,...,...,...,...,...
4995,4996.0,Gracious!,Gracious,17 July 1970,"Progressive Rock, Symphonic Prog",NaN,3.69,565,32
4996,4997.0,Knights of the Cross,Grave Digger,18 May 1998,"Power Metal, Heavy Metal","history, male vocals, heavy, concept album, an...",3.67,562,11
4997,4998.0,(Listen For) The Rag and Bone Man,And Also the Trees,12 November 2007,Art Rock,"atmospheric, poetic, dark, male vocals, ominou...",3.68,524,15
4998,4999.0,Hymnen für elektronische und konkrete Klänge,Studio für elektronische Musik des Westdeutsch...,1969,"Musique concrète, Modern Classical, Electronic","avant-garde, sampling, dense, complex, scary, ...",3.65,536,11


In [12]:
# format genres to put in lower case with no spaces, for use later when comparing words
new_rows = []
for index, row in df.iterrows():
    row['Genres'] = row['Genres'].split(',')
    row['Genres'] = [x.replace(' ','') for x in row['Genres']]
    row['Genres'] = [x.lower() for x in row['Genres']]
    new_rows.append(row)

df = pd.DataFrame(new_rows)

In [13]:
df

,Ranking,Album,Artist Name,Release Date,Genres,Descriptors,Average Rating,Number of Ratings,Number of Reviews
0,1.0,OK Computer,Radiohead,16 June 1997,"[alternativerock, artrock]","melancholic, anxious, futuristic, alienation, ...",4.23,"70,382",1531
1,2.0,Wish You Were Here,Pink Floyd,12 September 1975,"[progressiverock, artrock]","melancholic, atmospheric, progressive, male vo...",4.29,"48,662",983
2,3.0,In the Court of the Crimson King,King Crimson,10 October 1969,"[progressiverock, artrock]","fantasy, epic, progressive, philosophical, com...",4.30,"44,943",870
3,4.0,Kid A,Radiohead,3 October 2000,"[artrock, experimentalrock, electronic]","cold, melancholic, futuristic, atmospheric, an...",4.21,"58,590",734
4,5.0,To Pimp a Butterfly,Kendrick Lamar,15 March 2015,"[conscioushiphop, westcoasthiphop, jazzrap]","political, conscious, poetic, protest, concept...",4.27,"44,206",379
...,...,...,...,...,...,...,...,...,...
4995,4996.0,Gracious!,Gracious,17 July 1970,"[progressiverock, symphonicprog]",NaN,3.69,565,32
4996,4997.0,Knights of the Cross,Grave Digger,18 May 1998,"[powermetal, heavymetal]","history, male vocals, heavy, concept album, an...",3.67,562,11
4997,4998.0,(Listen For) The Rag and Bone Man,And Also the Trees,12 November 2007,[artrock],"atmospheric, poetic, dark, male vocals, ominou...",3.68,524,15
4998,4999.0,Hymnen für elektronische und konkrete Klänge,Studio für elektronische Musik des Westdeutsch...,1969,"[musiqueconcrète, modernclassical, electronic]","avant-garde, sampling, dense, complex, scary, ...",3.65,536,11


In [14]:
len(df)

5000

In [15]:
# drops the rows in Descriptors column with NaN as a value or a null value
df.dropna(subset=['Descriptors'], inplace=True)
len(df)

4886

In [16]:
# reformat the descriptors column for use in comparing words later
new_rows = []
for index, row in df.iterrows():
    row['Descriptors'] = row['Descriptors'].split(',')
    row['Descriptors'] = [x.replace(' ','') for x in row['Descriptors']]
    row['Descriptors'] = [x.lower() for x in row['Descriptors']]
    new_rows.append(row)

df = pd.DataFrame(new_rows)
df

,Ranking,Album,Artist Name,Release Date,Genres,Descriptors,Average Rating,Number of Ratings,Number of Reviews
0,1.0,OK Computer,Radiohead,16 June 1997,"[alternativerock, artrock]","[melancholic, anxious, futuristic, alienation,...",4.23,"70,382",1531
1,2.0,Wish You Were Here,Pink Floyd,12 September 1975,"[progressiverock, artrock]","[melancholic, atmospheric, progressive, malevo...",4.29,"48,662",983
2,3.0,In the Court of the Crimson King,King Crimson,10 October 1969,"[progressiverock, artrock]","[fantasy, epic, progressive, philosophical, co...",4.30,"44,943",870
3,4.0,Kid A,Radiohead,3 October 2000,"[artrock, experimentalrock, electronic]","[cold, melancholic, futuristic, atmospheric, a...",4.21,"58,590",734
4,5.0,To Pimp a Butterfly,Kendrick Lamar,15 March 2015,"[conscioushiphop, westcoasthiphop, jazzrap]","[political, conscious, poetic, protest, concep...",4.27,"44,206",379
...,...,...,...,...,...,...,...,...,...
4994,4995.0,Passion,Catharsis,1999,"[crustpunk, metalcore]","[malevocals, anarchism, aggressive, angry, hea...",3.65,616,6
4996,4997.0,Knights of the Cross,Grave Digger,18 May 1998,"[powermetal, heavymetal]","[history, malevocals, heavy, conceptalbum, ant...",3.67,562,11
4997,4998.0,(Listen For) The Rag and Bone Man,And Also the Trees,12 November 2007,[artrock],"[atmospheric, poetic, dark, malevocals, ominou...",3.68,524,15
4998,4999.0,Hymnen für elektronische und konkrete Klänge,Studio für elektronische Musik des Westdeutsch...,1969,"[musiqueconcrète, modernclassical, electronic]","[avant-garde, sampling, dense, complex, scary,...",3.65,536,11


In [19]:
# reformat the artist column for use in comparing words later
new_rows = []
for index, row in df.iterrows():
    row['Artist Name'] = row['Artist Name'].replace(" ", "").lower()
    new_rows.append(row)

df = pd.DataFrame(new_rows)
df.head(10)

,Ranking,Album,Artist Name,Release Date,Genres,Descriptors,Average Rating,Number of Ratings,Number of Reviews
0,1.0,OK Computer,radiohead,16 June 1997,"[alternativerock, artrock]","[melancholic, anxious, futuristic, alienation,...",4.23,"70,382",1531
1,2.0,Wish You Were Here,pinkfloyd,12 September 1975,"[progressiverock, artrock]","[melancholic, atmospheric, progressive, malevo...",4.29,"48,662",983
2,3.0,In the Court of the Crimson King,kingcrimson,10 October 1969,"[progressiverock, artrock]","[fantasy, epic, progressive, philosophical, co...",4.30,"44,943",870
3,4.0,Kid A,radiohead,3 October 2000,"[artrock, experimentalrock, electronic]","[cold, melancholic, futuristic, atmospheric, a...",4.21,"58,590",734
4,5.0,To Pimp a Butterfly,kendricklamar,15 March 2015,"[conscioushiphop, westcoasthiphop, jazzrap]","[political, conscious, poetic, protest, concep...",4.27,"44,206",379
5,6.0,Loveless,mybloodyvalentine,4 November 1991,"[shoegaze, noisepop]","[noisy, ethereal, atmospheric, romantic, dense...",4.24,"49,887",1223
6,7.0,The Dark Side of the Moon,pinkfloyd,23 March 1973,"[artrock, progressiverock]","[philosophical, atmospheric, introspective, ex...",4.20,"57,622",1549
7,8.0,Abbey Road,thebeatles,26 September 1969,[poprock],"[melodic, warm, malevocals, bittersweet, summe...",4.25,"44,544",961
8,9.0,The Velvet Underground & Nico,thevelvetunderground&nico,12 March 1967,"[artrock, experimentalrock]","[drugs, sexual, raw, urban, noisy, nihilistic,...",4.23,"45,570",929
9,10.0,The Rise and Fall of Ziggy Stardust and the Sp...,davidbowie,16 June 1972,"[glamrock, poprock]","[sciencefiction, melodic, anthemic, conceptalb...",4.26,"39,501",721


In [20]:
# drop uneeded columns
df.drop('Number of Reviews', axis=1, inplace=True)
df.drop('Number of Ratings', axis=1, inplace=True)
df.drop('Average Rating', axis=1, inplace=True)
df.drop('Ranking', axis=1, inplace=True)
df.drop('Release Date', axis=1, inplace=True)

df

,Album,Artist Name,Genres,Descriptors
0,OK Computer,radiohead,"[alternativerock, artrock]","[melancholic, anxious, futuristic, alienation,..."
1,Wish You Were Here,pinkfloyd,"[progressiverock, artrock]","[melancholic, atmospheric, progressive, malevo..."
2,In the Court of the Crimson King,kingcrimson,"[progressiverock, artrock]","[fantasy, epic, progressive, philosophical, co..."
3,Kid A,radiohead,"[artrock, experimentalrock, electronic]","[cold, melancholic, futuristic, atmospheric, a..."
4,To Pimp a Butterfly,kendricklamar,"[conscioushiphop, westcoasthiphop, jazzrap]","[political, conscious, poetic, protest, concep..."
...,...,...,...,...
4994,Passion,catharsis,"[crustpunk, metalcore]","[malevocals, anarchism, aggressive, angry, hea..."
4996,Knights of the Cross,gravedigger,"[powermetal, heavymetal]","[history, malevocals, heavy, conceptalbum, ant..."
4997,(Listen For) The Rag and Bone Man,andalsothetrees,[artrock],"[atmospheric, poetic, dark, malevocals, ominou..."
4998,Hymnen für elektronische und konkrete Klänge,studiofürelektronischemusikdeswestdeutschenrun...,"[musiqueconcrète, modernclassical, electronic]","[avant-garde, sampling, dense, complex, scary,..."


In [21]:
# remove the index and replace with the album column
df.set_index('Album', inplace=True)

df

,Artist Name,Genres,Descriptors
Album,,,
OK Computer,radiohead,"[alternativerock, artrock]","[melancholic, anxious, futuristic, alienation,..."
Wish You Were Here,pinkfloyd,"[progressiverock, artrock]","[melancholic, atmospheric, progressive, malevo..."
In the Court of the Crimson King,kingcrimson,"[progressiverock, artrock]","[fantasy, epic, progressive, philosophical, co..."
Kid A,radiohead,"[artrock, experimentalrock, electronic]","[cold, melancholic, futuristic, atmospheric, a..."
To Pimp a Butterfly,kendricklamar,"[conscioushiphop, westcoasthiphop, jazzrap]","[political, conscious, poetic, protest, concep..."
...,...,...,...
Passion,catharsis,"[crustpunk, metalcore]","[malevocals, anarchism, aggressive, angry, hea..."
Knights of the Cross,gravedigger,"[powermetal, heavymetal]","[history, malevocals, heavy, conceptalbum, ant..."
(Listen For) The Rag and Bone Man,andalsothetrees,[artrock],"[atmospheric, poetic, dark, malevocals, ominou..."


In [22]:
# create a descriptor_words column and place all of the words from other columns in that column
df['descriptor_words'] = ''
columns = df.columns
new_rows = []
for index, row in df.iterrows():
    words = ''
    for col in columns:
        if col != 'Artist Name':
            words = words + ' ' + ' '.join(row[col])
        else:
            words = words + ' ' + row[col]+ ' '
    row['descriptor_words'] = words
    new_rows.append(row)

df = pd.DataFrame(new_rows)
df

,Artist Name,Genres,Descriptors,descriptor_words
OK Computer,radiohead,"[alternativerock, artrock]","[melancholic, anxious, futuristic, alienation,...",radiohead alternativerock artrock melancholi...
Wish You Were Here,pinkfloyd,"[progressiverock, artrock]","[melancholic, atmospheric, progressive, malevo...",pinkfloyd progressiverock artrock melancholi...
In the Court of the Crimson King,kingcrimson,"[progressiverock, artrock]","[fantasy, epic, progressive, philosophical, co...",kingcrimson progressiverock artrock fantasy ...
Kid A,radiohead,"[artrock, experimentalrock, electronic]","[cold, melancholic, futuristic, atmospheric, a...",radiohead artrock experimentalrock electroni...
To Pimp a Butterfly,kendricklamar,"[conscioushiphop, westcoasthiphop, jazzrap]","[political, conscious, poetic, protest, concep...",kendricklamar conscioushiphop westcoasthipho...
...,...,...,...,...
Passion,catharsis,"[crustpunk, metalcore]","[malevocals, anarchism, aggressive, angry, hea...",catharsis crustpunk metalcore malevocals ana...
Knights of the Cross,gravedigger,"[powermetal, heavymetal]","[history, malevocals, heavy, conceptalbum, ant...",gravedigger powermetal heavymetal history ma...
(Listen For) The Rag and Bone Man,andalsothetrees,[artrock],"[atmospheric, poetic, dark, malevocals, ominou...",andalsothetrees artrock atmospheric poetic d...
Hymnen für elektronische und konkrete Klänge,studiofürelektronischemusikdeswestdeutschenrun...,"[musiqueconcrète, modernclassical, electronic]","[avant-garde, sampling, dense, complex, scary,...",studiofürelektronischemusikdeswestdeutschenru...


In [23]:
# drop other columns
df.drop('Artist Name', axis=1, inplace=True)
df.drop('Genres', axis=1, inplace=True)
df.drop('Descriptors', axis=1, inplace=True)

df

,descriptor_words
OK Computer,radiohead alternativerock artrock melancholi...
Wish You Were Here,pinkfloyd progressiverock artrock melancholi...
In the Court of the Crimson King,kingcrimson progressiverock artrock fantasy ...
Kid A,radiohead artrock experimentalrock electroni...
To Pimp a Butterfly,kendricklamar conscioushiphop westcoasthipho...
...,...
Passion,catharsis crustpunk metalcore malevocals ana...
Knights of the Cross,gravedigger powermetal heavymetal history ma...
(Listen For) The Rag and Bone Man,andalsothetrees artrock atmospheric poetic d...
Hymnen für elektronische und konkrete Klänge,studiofürelektronischemusikdeswestdeutschenru...


In [24]:
# instantiating and generating the count matrix
count = CountVectorizer()
count_matrix = count.fit_transform(df['descriptor_words'])
count_matrix

<4886x3935 sparse matrix of type '<class 'numpy.int64'>'
	with 59117 stored elements in Compressed Sparse Row format>

In [25]:
# generating the cosine similarity matrix
# compares with other entries from the same data
cosine_sim = cosine_similarity(count_matrix, count_matrix)
cosine_sim

array([[1.        , 0.38461538, 0.23076923, ..., 0.24019223, 0.        ,
        0.        ],
       [0.38461538, 1.        , 0.38461538, ..., 0.24019223, 0.        ,
        0.07692308],
       [0.23076923, 0.38461538, 1.        , ..., 0.24019223, 0.13867505,
        0.        ],
       ...,
       [0.24019223, 0.24019223, 0.24019223, ..., 1.        , 0.14433757,
        0.        ],
       [0.        , 0.        , 0.13867505, ..., 0.14433757, 1.        ,
        0.        ],
       [0.        , 0.07692308, 0.        , ..., 0.        , 0.        ,
        1.        ]])

In [26]:
# creating a series for the movie titles so they are associated to an ordered numerical
# list I will use later to match the indexes
indices = pd.Series(df.index)
indices[:20]

0                                           OK Computer
1                                    Wish You Were Here
2                      In the Court of the Crimson King
3                                                 Kid A
4                                   To Pimp a Butterfly
5                                              Loveless
6                             The Dark Side of the Moon
7                                            Abbey Road
8                         The Velvet Underground & Nico
9     The Rise and Fall of Ziggy Stardust and the Sp...
10                                             Revolver
11                                          Madvillainy
12                                      Remain in Light
13                  The Black Saint and the Sinner Lady
14                                          In Rainbows
15                                       A Love Supreme
16                               good kid, m.A.A.d city
17                                             P

In [53]:
# function that takes in the album title as input and returns the top 10 recommended albums
def recommendations(title, cosine_sim = cosine_sim):

    recommended_albums = []

    # getting the index of the movie that matches the title
    idx = indices[indices == title].index[0]
    
    # creating a Series with the similarity scores in descending order
    score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)
    
    # getting the indexes of the 10 most similiar movies
    top_10_indexes = list(score_series.iloc[1:11].index)
    print(top_10_indexes)

    for i in top_10_indexes:
        recommended_albums.append(list(df.index)[i])

    return recommended_albums

# input the album name below to get 10 albums that are similar
albums = recommendations('To Pimp a Butterfly')

[2592, 16, 3465, 3496, 4337, 2554, 466, 3291, 431, 246]


In [54]:
# displays the artist and album name together
new_df = pd.read_csv("Top5000Albums.csv")
new_rows = []
for album in albums:
    for index, row in new_df.iterrows():
        if row['Album'] == album:
            print(f"{album} by {row['Artist Name']}")
    

Untitled Unmastered. by Kendrick Lamar
good kid, m.A.A.d city by Kendrick Lamar
Section.80 by Kendrick Lamar
Lupe Fiasco's Food & Liquor by Lupe Fiasco
Damn. by Kendrick Lamar
Pequeñas anécdotas sobre las instituciones by Sui Generis
Stankonia by OutKast
s.o.n.o.g.r.a.m. by One Be Lo
Pieces of a Man by Gil Scott-Heron
Black on Both Sides by Mos Def
